In [18]:
from PIL import Image
import numpy as np

def circle(h,w,centre=[964,1110],radius=80):
    Y,X=np.ogrid[:h,:w]
    distance=np.sqrt((Y-centre[0])**2+(X-centre[1])**2)<=radius
    return distance

def convolution(img,kernel):
    img_row,img_column,img_channel=image.shape
    #kernel_size=kernel.shape[0]
    #padding_width=(kernel_size-1)//2
    output=np.zeros(img.shape)
    padded_image=np.zeros((img_row+kernel.shape[0]-1,img_column+kernel.shape[1]-1,img_channel))
    padded_image[kernel.shape[0]-1:kernel.shape[0]-1+img_row,kernel.shape[1]-1:kernel.shape[1]-1+img_column]=img
    #padded_image[0,0,:]=image[0,0,:]
   # padded_image[-1,-1,:]=image[-1,-1,:]
    for j in range(img_column):
        for i in range(img_row):
            for k in range(img_channel):
                s=np.sum(padded_image[i:i+kernel.shape[0],j:j+kernel.shape[1],k]*kernel)
                output[i,j,k]=s
    if img.shape[2]==4:
        output[:,:,3]=img[:,:,3]
    return output

gaussian=np.array([[1,  4,  6,  4, 1],
                         [4, 16, 24, 16, 4],
                         [6, 24, 36, 24, 6],
                         [4, 16, 24, 16, 4],
                         [1,  4,  6,  4, 1]])/256

image=np.array(Image.open('roimessi.jpg'))
image=convolution(image,gaussian)
mask=circle(image.shape[0],image.shape[1])
masked=image.copy()
masked[~mask]=0
r_shift=-25
c_shift=-670

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if masked[i,j,:].all()!=0:
            masked[i+r_shift,j+c_shift,:]=masked[i,j,:]
            masked[i,j,:]=0
            image[i+r_shift,j+c_shift,:]=0
image=np.bitwise_or(image.astype(int),masked.astype(int))
pil_img=Image.fromarray(image.astype(np.uint8))
pil_img.show()
            
        